In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow import convert_to_tensor

VERBOSE = 1  # Set verbose to 1 to see training progress

In [ ]:
def get_model():
    model = Sequential()
    model.add(LSTM(50, return_sequences=False, input_shape=(None, 1))) # This adds an LSTM layer with 50 neurons (units)
    model.add(Dropout(0.2))
    model.add(Dense(1))

    model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error', metrics=['mean_absolute_error'])
    return model

In order to use preprocessed dataset with only LCLid (unique consumer ID) and KWH/hh (per hour) (energy consumption per hour) columns replace the following cell with the code snipped below:

hourly_data = pd.read_csv("Preprocessed_data2013_2.csv", dtype={'LCLid': np.int16, 'KWH/hh (per hour) ': np.float64})


In [ ]:
hourly_data = pd.read_csv("Preprocessed_data.csv", dtype={'LCLid': np.int16, 'KWH/hh (per hour) ': np.float64, 'dayoftheyear': np.int16,
       'hour': np.int8, 'is_weekend': np.int8})



In [ ]:
# Define number of meters to be used in the project, replace the  hourly_data['LCLid'].max() with another number (e.g. NUM_OF_METERS = 100) to reduce
# number of meters used.
NUM_OF_METERS = hourly_data['LCLid'].max()


In [ ]:
# Function to split data into train, validation, and test sets
def train_test_validate(data):

    for i in range(0, NUM_OF_METERS):

        
        tmp_data = data[data['LCLid'] == i] # Get the data for current meter

        
        val_split = int(len(tmp_data) * 0.8)
        test_split = int(len(tmp_data) * 0.9)

        # Set initial splits for current meter
        train_ = tmp_data[:val_split]
        vali_ = tmp_data[val_split:test_split]
        test_ = tmp_data[test_split:]

        # Concatanate the test data 
        if (i > 0):
            train = pd.concat([train, train_], ignore_index=True)
            valid = pd.concat([valid, vali_], ignore_index=True)
            test = pd.concat([test, test_], ignore_index=True)
        else:
            train = train_
            valid = vali_
            test = test_

    return {"train": train, "test": test, "validation": valid}

dataset_splits = train_test_validate(hourly_data)

# Convert data to tensors
x_train_tensor = tf.convert_to_tensor(np.asarray(dataset_splits["train"].drop(columns=['KWH/hh (per hour) '])))
y_train_tensor = tf.convert_to_tensor(np.asarray(dataset_splits["train"]['KWH/hh (per hour) ']))
x_val_tensor = tf.convert_to_tensor(np.asarray(dataset_splits["validation"].drop(columns=['KWH/hh (per hour) '])))
y_val_tensor = tf.convert_to_tensor(np.asarray(dataset_splits["validation"]['KWH/hh (per hour) ']))
x_test_tensor = tf.convert_to_tensor(np.asarray(dataset_splits["test"].drop(columns=['KWH/hh (per hour) '])))
y_test_tensor = tf.convert_to_tensor(np.asarray(dataset_splits["test"]['KWH/hh (per hour) ']))

In [ ]:
# Initialize and train the model
model = get_model()
history = model.fit(x_train_tensor, y_train_tensor, validation_data=(x_val_tensor, y_val_tensor), epochs=10, batch_size=512, verbose=VERBOSE)


In [ ]:
# Evaluate the model
results = model.evaluate(x_test_tensor, y_test_tensor, verbose=VERBOSE)
print(f"Test MSE: {results[0]}, Test MAE: {results[1]}")

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

rounds = np.arange(0,len(history.history['loss']))
loss = history.history['loss']
acc = history.history['mean_absolute_error']

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                    subplot_titles=("MEAN ABSOLUTE ERROR", "MEAN SQUARED ERROR"))

# Add scatter plot for accuracy
fig.add_trace(go.Scatter(x=rounds, y=acc, mode='markers', name='MAE'), row=1, col=1)
# Add line plot for accuracy
fig.add_trace(go.Scatter(x=rounds, y=acc, mode='lines', name='MAE Line'), row=1, col=1)

# Add scatter plot for loss
fig.add_trace(go.Scatter(x=rounds, y=loss, mode='markers', name='MSE'), row=2, col=1)
# Add line plot for loss
fig.add_trace(go.Scatter(x=rounds, y=loss, mode='lines', name='MSE Line'), row=2, col=1)

# Update layout
fig.update_layout(
    height=800,  # Height of the figure
    title_text="Centralized Model",
)

# Update x-axis for all subplots
fig.update_xaxes(title_text="Round", row=2, col=1)
# Update y-axis for each subplot
fig.update_yaxes(title_text="MEAN ABSOLUTE ERROR", row=1, col=1)
fig.update_yaxes(title_text="MEAN SQUARED ERROR", row=2, col=1)

# Show the plot
fig.show()